In [36]:
import pandas as pd
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify
import flask
#!pip install flask_ngrok

In [95]:
df = pd.read_csv("address_with_coords.csv").dropna()
df["lon"] = df.point.apply(lambda x: float(x.split()[0]))
df["lat"] = df.point.apply(lambda x: float(x.split()[1]))
df = df[(df["lat"]>55.582238) & (df["lat"]<55.905471) & (df["lon"]>37.372692) & (df["lon"]<37.851569)]
df = df[["timestamp", "lon", "lat"]]

df["timestamp"] = df.timestamp.apply(lambda x: pd.to_datetime(x, format="%Y-%m-%dT%H:%M:%S.%f"))
df["minute"] = df.timestamp.apply(lambda x: x.minute)
df["second"] = df.timestamp.apply(lambda x: x.second)
df["hour"] = df.timestamp.apply(lambda x: x.hour)
df["month"] = df.timestamp.apply(lambda x: x.month)
df["day"] = df.timestamp.apply(lambda x: x.day)
df["year"] = df.timestamp.apply(lambda x: x.year)
df["microsecond"] = df.timestamp.apply(lambda x: x.microsecond)
df = df.drop("timestamp", axis=1)
df.to_csv("mini_address_with_coords.csv", index=False)

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/")
def home():
    points = []
    for i in range(int(flask.request.args.get("points"))):
      new_df = pd.read_csv("mini_address_with_coords.csv").sample(30000)

      #lat = new_df.lat.mean()
      #lon = new_df.lon.mean()

      regr = RandomForestRegressor(max_depth=2)
      regr.fit(new_df.drop("lon", axis=1), new_df["lon"])
      lon = regr.predict(pd.read_csv("mini_address_with_coords.csv").sample(1).drop("lon", axis=1).to_numpy())[0]

      regr = RandomForestRegressor(max_depth=2)
      regr.fit(new_df.drop("lat", axis=1), new_df["lat"])
      lat = regr.predict(pd.read_csv("mini_address_with_coords.csv").sample(1).drop("lat", axis=1).to_numpy())[0]

      points.append({"lat": lat, "lon": lon})

    return jsonify(points)
  
app.run()